# OpenLineage-Spark Demo


Run the following cells and open Marquez at http://127.0.0.1:3000/

### To-dos

- how do you add facets and additional details (e.g., the pyspark code, the source csv filename, etc.?
- how to you set the namespace for the events? While we have `.config('spark.openlineage.namespace', 'spark_integration')` for the code, the datasets show up in the `files` namespace in Marquez. 

In [1]:
from pyspark.sql import SparkSession
import urllib.request

# Set these to your own project and bucket
spark = (SparkSession.builder.master('local').appName('openlineage_spark_demo')
             # Install and set up the OpenLineage listener
             #.config('spark.jars.packages', 'io.openlineage:openlineage-spark:0.3.+')
             .config('spark.jars.packages', 'io.openlineage:openlineage-spark:0.6.0')
             .config('spark.extraListeners', 'io.openlineage.spark.agent.OpenLineageSparkListener')
             .config('spark.openlineage.host', 'http://marquez_api:5000')
             .config('spark.openlineage.namespace', 'spark_integration')
             .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.openlineage#openlineage-spark added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-383cfea6-3cfd-4622-8b6b-d4781fd6bc4a;1.0
	confs: [default]
	found io.openlineage#openlineage-spark;0.6.0 in central
	found io.openlineage#openlineage-java;0.6.0 in central
	found org.javassist#javassist;3.27.0-GA in central
	found org.apache.httpcomponents.client5#httpclient5;5.0.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.0.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.0.2 in central
	found org.slf4j#slf4j-api;1.7.25 in central
	found commons-codec#commons-codec;1.13 in central
	found com.fasterxml.jackson.datatype#jackson-datatype-jsr310;2.6.7 in central
:: resolution report :: resolve 422ms :: artifacts dl 20ms
	:: modules in use:
	com.fasterxml.jackson.datatype#jackson-datatype-jsr310;2.6.7 from central in [defau

# Testing Local Read

In [2]:
# Looks like spark.read doesn't trigger an OpenLineage event? Shouldn't we track reads as well?
df = spark.read.format("csv").option("header",True).load("test.csv")
df.show()

22/04/02 02:08:44 ERROR EventEmitter: Could not emit lineage: {"eventType":"START","eventTime":"2022-04-02T02:08:42.847Z","run":{"runId":"41ead379-b1fe-495c-a934-a8cc30fe1459","facets":{"spark_version":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","spark-version":"3.1.2","openlineage-spark-version":"0.6.0"},"spark.logicalPlan":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","plan":[{"class":"org.apache.spark.sql.catalyst.plans.logical.GlobalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.Literal","num-children":0,"value":"1","dataType":"integer"}],"child":0},{"class":"org.apache.spark.sql.catalyst.plans.logical.LocalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.Li

22/04/02 02:08:44 ERROR EventEmitter: Could not emit lineage: {"eventType":"COMPLETE","eventTime":"2022-04-02T02:08:44.67Z","run":{"runId":"41ead379-b1fe-495c-a934-a8cc30fe1459","facets":{"spark_version":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","spark-version":"3.1.2","openlineage-spark-version":"0.6.0"},"spark.logicalPlan":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","plan":[{"class":"org.apache.spark.sql.catalyst.plans.logical.GlobalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.Literal","num-children":0,"value":"1","dataType":"integer"}],"child":0},{"class":"org.apache.spark.sql.catalyst.plans.logical.LocalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.

+--------------------+--------------------+-------+-------+---------+
|          dependency|             package|version|   type|important|
+--------------------+--------------------+-------+-------+---------+
|          @babel/cli|benchmark/package...| ^7.8.4|tooling|     null|
|          @babel/cli|benchmark/package...| ^7.8.4|tooling|     null|
|          @babel/cli|asset-library/pac...| ^7.8.4|tooling|     null|
|         @babel/core|benchmark/package...| ^7.8.4|tooling|      YES|
|         @babel/core|benchmark/package...| ^7.8.4|tooling|     null|
|         @babel/core|benchmark/package...| ^7.9.0|tooling|     null|
|         @babel/core|asset-library/pac...| ^7.8.4|tooling|     null|
|         @babel/core|asset-library/pac...| ^7.9.6|tooling|     null|
|         @babel/core|adc/packages/asse...|^7.12.3|tooling|     null|
|         @babel/core|  adc/packages/types|^7.12.0|tooling|     null|
|         @babel/core|adc/packages/stud...|^7.12.0|tooling|     null|
|         @babel/cor

22/04/02 02:08:45 ERROR EventEmitter: Could not emit lineage: {"eventType":"START","eventTime":"2022-04-02T02:08:44.944Z","run":{"runId":"642afaee-db37-480e-b529-1dce80b6eba9","facets":{"spark_version":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","spark-version":"3.1.2","openlineage-spark-version":"0.6.0"},"spark.logicalPlan":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","plan":[{"class":"org.apache.spark.sql.catalyst.plans.logical.GlobalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.Literal","num-children":0,"value":"21","dataType":"integer"}],"child":0},{"class":"org.apache.spark.sql.catalyst.plans.logical.LocalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.L

22/04/02 02:08:45 ERROR EventEmitter: Could not emit lineage: {"eventType":"COMPLETE","eventTime":"2022-04-02T02:08:45.208Z","run":{"runId":"642afaee-db37-480e-b529-1dce80b6eba9","facets":{"spark_version":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","spark-version":"3.1.2","openlineage-spark-version":"0.6.0"},"spark.logicalPlan":{"_producer":"https://github.com/OpenLineage/OpenLineage/tree/0.6.0/integration/spark","_schemaURL":"https://openlineage.io/spec/1-0-2/OpenLineage.json#/$defs/RunFacet","plan":[{"class":"org.apache.spark.sql.catalyst.plans.logical.GlobalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expressions.Literal","num-children":0,"value":"21","dataType":"integer"}],"child":0},{"class":"org.apache.spark.sql.catalyst.plans.logical.LocalLimit","num-children":1,"limitExpr":[{"class":"org.apache.spark.sql.catalyst.expression

In [ ]:
# note that this triggers a OpenLineage event with data source as "Notebooks", as opposed to test.csv
# seem to be a bug

df.groupBy('dependency').count().write.option("header",True).mode("overwrite").csv("dependency_count.csv")


In [ ]:
# what if we break this op up
df1 = df.groupBy('dependency').count()
df1.write.option("header",True).mode("overwrite").csv("dependency_count2.csv")


In [ ]:
df1.write.option("header",True).mode("overwrite").csv("dependency_count3.csv")


In [ ]:
rownum = df.count()
spark.sparkContext.parallelize([rownum]).coalesce(1).saveAsTextFile("rownum.txt")

# Junk code below

In [ ]:
type(None)

In [ ]:
df.dtypes

In [ ]:
df.schema.fields

In [ ]:
from pyspark.sql import DataFrame
from pyspark.rdd import RDD

def foo(x):
    if isinstance(x, RDD):
        return "RDD"
    if isinstance(x, DataFrame):
        return "DataFrame"

print(foo(spark.sparkContext.parallelize([])))
## 'RDD'
print(foo(spark.sparkContext.parallelize([("foo", 1)]).toDF()))
## 'DataFrame'

#isinstance(df, "pyspark.sql.dataframe.DataFrame")

In [ ]:
!pip install marquez-python

In [ ]:
from marquez_client import MarquezClient

client = MarquezClient(url='http://marquez_api:5000')

# list namespaces
client.list_namespaces()


In [ ]:
!pip install openlineage-python

In [ ]:
from openlineage.client.client import OpenLineageClient
from openlineage.client.run import RunEvent, RunState, Run, Job
from unittest.mock import MagicMock

session = MagicMock()
client = OpenLineageClient(url="http://192.168.86.39:5000", session=None)

client.emit(
    RunEvent(
        RunState.START,
        "2020-01-01",
        Run("69f4acab-b87d-4fc0-b27b-8ea950370ff3"),
        Job("openlineage", "job"),
        "producer"
    )
)

In [ ]:
session.post.assert_called_with(
    "http://marquez-api2:5000/api/v1/lineage",
    '{"eventTime": "2020-01-01", "eventType": "START", "inputs": [], "job": '
    '{"facets": {}, "name": "job", "namespace": "openlineage"}, "outputs": [], '
    '"producer": "producer", "run": {"facets": {}, "runId": '
    '"69f4acab-b87d-4fc0-b27b-8ea950370ff3"}}',
    timeout=5.0,
    verify=True
)

In [ ]:
from openlineage.client import constants
from openlineage.client.run import RunEvent
from openlineage.client.serde import Serde

import requests
url = 'http://192.168.86.39:5000/api/v1/lineage'
headers = {"charset": "utf-8", "Content-Type": "application/json"}


In [ ]:
data

In [ ]:
data = Serde.to_json(RunEvent(
        RunState.START,
        "2020-01-01",
        Run("69f4acab-b87d-4fc0-b27b-8ea950370ff3"),
        Job("openlineage", "job"),
        "producer"
    ))
r = requests.post(url, data=data, headers=headers)
r.text

In [ ]:
data = """{
        "eventType": "START",
        "eventTime": "2020-12-28T19:52:00.001+10:00",
        "run": {
          "runId": "d46e465b-d358-4d32-83d4-df660ff614dd"
        },
        "job": {
          "namespace": "gary-namespace",
          "name": "my-job"
        },
        "inputs": [{
          "namespace": "gary-namespace",
          "name": "my-input"
        }],  
        "producer": "https://github.com/OpenLineage/OpenLineage/blob/v1-0-0/client"
      }"""
r = requests.post(url, data=data, headers=headers)
r.text

In [ ]:
data2 = """{
        "eventType": "COMPLETE",
        "eventTime": "2020-12-28T20:52:00.001+10:00",
        "run": {
          "runId": "d46e465b-d358-4d32-83d4-df660ff614dd"
        },
        "job": {
          "namespace": "gary-namespace",
          "name": "my-job"
        },
        "outputs": [{
          "namespace": "gary-namespace",
          "name": "my-output",
          "facets": {
            "schema": {
              "_producer": "https://github.com/OpenLineage/OpenLineage/blob/v1-0-0/client",
              "_schemaURL": "https://github.com/OpenLineage/OpenLineage/blob/v1-0-0/spec/OpenLineage.json#/definitions/SchemaDatasetFacet",
              "fields": [
                { "name": "a", "type": "VARCHAR"},
                { "name": "b", "type": "VARCHAR"}
              ]
            }
          }
        }],     
        "producer": "https://github.com/OpenLineage/OpenLineage/blob/v1-0-0/client"
      }"""

r = requests.post(url, data=data2, headers=headers)
r.text